# CS572 Project
* We selected the paper: {}
* Based on the data and model of given paper, we are planning to make an extension of LSTM that predicts {}.

## Preparing

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import time
# from matplotlib import pyplot as plt

data_path_train = 'Data/Training' # add path
data_path_test = 'Data/Testing'

input_size = 8
hidden_size = 128
num_layers = 3
num_epochs = 2000
batch_size = 32
learning_rate = 0.001
weight_decay=0.00001

Check if your computer is prepared to run pytorch model with CUDA.

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # x -> batch_size, seq, input_size
        self.fc = nn.Linear(hidden_size, 1)
        
#     def makehc_zero(self, batch_size):
#         self.hn = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
#         self.cn = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
#         print("clear")
        
    
    def forward(self, x):
#         if self.training == True:
#             # Set initial hidden and cell states 
#             self.makehc_zero(x.size(0))

        
        # Forward propagate LSTM
#         out, (self.hn, self.cn) = self.lstm(x, (self.hn, self.cn))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        out, (self.hm, self.cn) = self.lstm(x,(h0,c0))

        out_original = out
        out_original_fc = self.fc(out_original) # out_original : Tensor of shape (batch_size, seq_length, 1)
        
#         pred_y = self.fc(out[-1].view(x.size(1),-1)) # pred_y : tensor of shape (40,1), taking the last section of a batch
#         print("This is x.size(1):", x.size(1))
#         print("This is out[-1] :", out[-1].shape)
#         print(out[-1])
#         print("This is out[-1].view(x.size(1),-1):", out[-1].view(x.size(1),-1).shape)
#         print(out[-1].view(x.size(1),-1))
#         print("This is pred_y : ", pred_y.shape)
#         print(pred_y)
        

#         out = out[:, -1, :]
#         print("This is output right before linear : ", out.shape)
#         out = self.fc(out)
#         print("This is output after linear : ", out.shape)
#         print(out)
        
        return out_original_fc

Data Loading

In [36]:
# Training and Testing: To train fingertip forces from input
# data, a three-layer BT-LSTM with 128 hidden units is used.
# Sequence time length of BT-LSTM was set to T = 40. The final
# output of the BT-LSTM layer is fed into the final fully-connected
# layer to obtain the one-dimensional output. The mean squared
# error (MSE) function is used to measure the loss between the
# predicted and the ground truth contact force. During training,
# the number of epoch was 2000, and the mini batch size was 32.
# Adam optimizer was used with 0.001 learning rate and 10−5
# weight decay
lstm = LSTM(input_size, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=weight_decay)

Load cell if needed

In [ ]:
lstm.load_state_dict(torch.load('model.ckpt'))

Train model

In [42]:
for epoch in range (2000):
    for path_idx in os.listdir(data_path_train):
        cur_path = os.path.join(data_path_train, path_idx)
        print(cur_path)
        input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
        # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
        inputs = torch.Tensor(input_file)
    #     print(inputs[0])

        inputs = inputs[:len(inputs)-len(inputs)%40]
    #     labels = inputs[:, [5]]
    # #     print(labels.shape)
    #     labels = labels.view([-1, 40, 1])

        inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
        inputs = inputs.view([-1,40,9])

        train_loader = torch.utils.data.DataLoader(dataset=inputs,
                                               batch_size=batch_size, 
                                               shuffle=True)
        # Train the model
        start = time.time() 
        for i, inputs in enumerate(train_loader):
            inputs, labels = torch.split(inputs, [8, 1], 2)
#             print("inputs shape : ", inputs.shape)
            inputs = inputs.to(device)
    #         labels = labels[:,[39],:].view([-1,1])
    #         labels = labels[-1,:,:].view([-1,1])
            labels = labels.to(device)
    #         print("labels shape : ", labels.shape)


            # Forward pass
            outputs = lstm(inputs)

    #         print(labels.shape)

            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % 10 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            print('time :', time.time() - start)
    print("training complete")

Data/Training\190722_tension_test_space_1_trial_2.csv
Data/Training\190722_tension_test_space_1_trial_3.csv
Data/Training\190722_tension_test_space_2_trial_1.csv
Data/Training\190722_tension_test_space_2_trial_2.csv
Data/Training\190722_tension_test_space_2_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_2.csv
Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\190722_tension_test_space_3_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_5.csv
Data/Training\190722_tension_test_space_3_trial_6.csv
Data/Training\190722_tension_test_space_4_trial_2.csv
Data/Training\190722_tension_test_space_4_trial_3.csv
Data/Training\190722_tension_test_space_5_trial_2.csv
Data/Training\190722_tension_test_space_5_trial_3.csv
Data/Training\test11thick_1.csv
Data/Training\test11thick_3.csv
Data/Training\test22thick_2.csv
Data/Training\test33thick_2.csv
Data/Training\test33thick_3.csv
Data/Training\test55thick_2.csv
Data/Training\test55thick_3.csv
training complete
Da

Data/Training\190722_tension_test_space_2_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_2.csv
Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\190722_tension_test_space_3_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_5.csv
Data/Training\190722_tension_test_space_3_trial_6.csv
Data/Training\190722_tension_test_space_4_trial_2.csv
Data/Training\190722_tension_test_space_4_trial_3.csv
Data/Training\190722_tension_test_space_5_trial_2.csv
Data/Training\190722_tension_test_space_5_trial_3.csv
Data/Training\test11thick_1.csv
Data/Training\test11thick_3.csv
Data/Training\test22thick_2.csv
Data/Training\test33thick_2.csv
Data/Training\test33thick_3.csv
Data/Training\test55thick_2.csv
Data/Training\test55thick_3.csv
training complete
Data/Training\190722_tension_test_space_1_trial_2.csv
Epoch [10/2000], Loss: 1.2258
time : 0.13713693618774414
Data/Training\190722_tension_test_space_1_trial_3.csv
Epoch [10/2000], Loss: 0.3265
time : 0.62133765220642

Data/Training\190722_tension_test_space_3_trial_2.csv
Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\190722_tension_test_space_3_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_5.csv
Data/Training\190722_tension_test_space_3_trial_6.csv
Data/Training\190722_tension_test_space_4_trial_2.csv
Data/Training\190722_tension_test_space_4_trial_3.csv
Data/Training\190722_tension_test_space_5_trial_2.csv
Data/Training\190722_tension_test_space_5_trial_3.csv
Data/Training\test11thick_1.csv
Data/Training\test11thick_3.csv
Data/Training\test22thick_2.csv
Data/Training\test33thick_2.csv
Data/Training\test33thick_3.csv
Data/Training\test55thick_2.csv
Data/Training\test55thick_3.csv
training complete
Data/Training\190722_tension_test_space_1_trial_2.csv
Data/Training\190722_tension_test_space_1_trial_3.csv
Data/Training\190722_tension_test_space_2_trial_1.csv
Data/Training\190722_tension_test_space_2_trial_2.csv
Data/Training\190722_tension_test_space_2_trial_4.csv
Da

Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\190722_tension_test_space_3_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_5.csv
Data/Training\190722_tension_test_space_3_trial_6.csv
Data/Training\190722_tension_test_space_4_trial_2.csv
Data/Training\190722_tension_test_space_4_trial_3.csv
Data/Training\190722_tension_test_space_5_trial_2.csv
Data/Training\190722_tension_test_space_5_trial_3.csv
Data/Training\test11thick_1.csv
Data/Training\test11thick_3.csv
Data/Training\test22thick_2.csv
Data/Training\test33thick_2.csv
Data/Training\test33thick_3.csv
Data/Training\test55thick_2.csv
Data/Training\test55thick_3.csv
training complete
Data/Training\190722_tension_test_space_1_trial_2.csv
Data/Training\190722_tension_test_space_1_trial_3.csv
Data/Training\190722_tension_test_space_2_trial_1.csv
Data/Training\190722_tension_test_space_2_trial_2.csv
Data/Training\190722_tension_test_space_2_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_2.csv
Da

Epoch [30/2000], Loss: 6.7520
time : 0.21941113471984863
Data/Training\test11thick_3.csv
Epoch [30/2000], Loss: 0.8951
time : 0.05086398124694824
Data/Training\test22thick_2.csv
Epoch [30/2000], Loss: 1.7284
time : 0.21641874313354492
Data/Training\test33thick_2.csv
Epoch [30/2000], Loss: 1.9396
time : 0.07878875732421875
Data/Training\test33thick_3.csv
Epoch [30/2000], Loss: 4.7249
time : 0.13064885139465332
Data/Training\test55thick_2.csv
Epoch [30/2000], Loss: 4.0851
time : 0.1436161994934082
Data/Training\test55thick_3.csv
Epoch [30/2000], Loss: 4.1590
time : 0.1824336051940918
training complete
Data/Training\190722_tension_test_space_1_trial_2.csv
Data/Training\190722_tension_test_space_1_trial_3.csv
Data/Training\190722_tension_test_space_2_trial_1.csv
Data/Training\190722_tension_test_space_2_trial_2.csv
Data/Training\190722_tension_test_space_2_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_2.csv
Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\1

...

Data/Training\test11thick_3.csv
Data/Training\test22thick_2.csv
Data/Training\test33thick_2.csv
Data/Training\test33thick_3.csv
Data/Training\test55thick_2.csv
Data/Training\test55thick_3.csv
training complete
Data/Training\190722_tension_test_space_1_trial_2.csv
Data/Training\190722_tension_test_space_1_trial_3.csv
Data/Training\190722_tension_test_space_2_trial_1.csv
Data/Training\190722_tension_test_space_2_trial_2.csv
Data/Training\190722_tension_test_space_2_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_2.csv
Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\190722_tension_test_space_3_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_5.csv
Data/Training\190722_tension_test_space_3_trial_6.csv
Data/Training\190722_tension_test_space_4_trial_2.csv
Data/Training\190722_tension_test_space_4_trial_3.csv
Data/Training\190722_tension_test_space_5_trial_2.csv
Data/Training\190722_tension_test_space_5_trial_3.csv
Data/Training\test11thick_1.csv
Da

Data/Training\test11thick_3.csv
Data/Training\test22thick_2.csv
Data/Training\test33thick_2.csv
Data/Training\test33thick_3.csv
Data/Training\test55thick_2.csv
Data/Training\test55thick_3.csv
training complete
Data/Training\190722_tension_test_space_1_trial_2.csv
Data/Training\190722_tension_test_space_1_trial_3.csv
Data/Training\190722_tension_test_space_2_trial_1.csv
Data/Training\190722_tension_test_space_2_trial_2.csv
Data/Training\190722_tension_test_space_2_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_2.csv
Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\190722_tension_test_space_3_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_5.csv
Data/Training\190722_tension_test_space_3_trial_6.csv
Data/Training\190722_tension_test_space_4_trial_2.csv
Data/Training\190722_tension_test_space_4_trial_3.csv
Data/Training\190722_tension_test_space_5_trial_2.csv
Data/Training\190722_tension_test_space_5_trial_3.csv
Data/Training\test11thick_1.csv
Da

Data/Training\test11thick_3.csv
Data/Training\test22thick_2.csv
Data/Training\test33thick_2.csv
Data/Training\test33thick_3.csv
Data/Training\test55thick_2.csv
Data/Training\test55thick_3.csv
training complete
Data/Training\190722_tension_test_space_1_trial_2.csv
Data/Training\190722_tension_test_space_1_trial_3.csv
Data/Training\190722_tension_test_space_2_trial_1.csv
Data/Training\190722_tension_test_space_2_trial_2.csv
Data/Training\190722_tension_test_space_2_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_2.csv
Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\190722_tension_test_space_3_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_5.csv
Data/Training\190722_tension_test_space_3_trial_6.csv
Data/Training\190722_tension_test_space_4_trial_2.csv
Data/Training\190722_tension_test_space_4_trial_3.csv
Data/Training\190722_tension_test_space_5_trial_2.csv
Data/Training\190722_tension_test_space_5_trial_3.csv
Data/Training\test11thick_1.csv
Da

Epoch [1990/2000], Loss: 1.0241
time : 0.21342921257019043
Data/Training\190722_tension_test_space_4_trial_2.csv
Epoch [1990/2000], Loss: 1.2836
time : 0.31316113471984863
Data/Training\190722_tension_test_space_4_trial_3.csv
Epoch [1990/2000], Loss: 0.9353
time : 0.37998390197753906
Data/Training\190722_tension_test_space_5_trial_2.csv
Epoch [1990/2000], Loss: 0.5988
time : 0.179518461227417
Data/Training\190722_tension_test_space_5_trial_3.csv
Epoch [1990/2000], Loss: 0.1678
time : 0.5644888877868652
Data/Training\test11thick_1.csv
Epoch [1990/2000], Loss: 0.5647
time : 0.22040772438049316
Data/Training\test11thick_3.csv
Epoch [1990/2000], Loss: 3.7246
time : 0.05983924865722656
Data/Training\test22thick_2.csv
Epoch [1990/2000], Loss: 0.3523
time : 0.21879363059997559
Data/Training\test33thick_2.csv
Epoch [1990/2000], Loss: 1.6350
time : 0.08672714233398438
Data/Training\test33thick_3.csv
Epoch [1990/2000], Loss: 2.0907
time : 0.14461278915405273
Data/Training\test55thick_2.csv
Epoch

Data/Training\190722_tension_test_space_1_trial_3.csv
Data/Training\190722_tension_test_space_2_trial_1.csv
Data/Training\190722_tension_test_space_2_trial_2.csv
Data/Training\190722_tension_test_space_2_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_2.csv
Data/Training\190722_tension_test_space_3_trial_3.csv
Data/Training\190722_tension_test_space_3_trial_4.csv
Data/Training\190722_tension_test_space_3_trial_5.csv
Data/Training\190722_tension_test_space_3_trial_6.csv
Data/Training\190722_tension_test_space_4_trial_2.csv
Data/Training\190722_tension_test_space_4_trial_3.csv
Data/Training\190722_tension_test_space_5_trial_2.csv
Data/Training\190722_tension_test_space_5_trial_3.csv
Data/Training\test11thick_1.csv
Data/Training\test11thick_3.csv
Data/Training\test22thick_2.csv
Data/Training\test33thick_2.csv
Data/Training\test33thick_3.csv
Data/Training\test55thick_2.csv
Data/Training\test55thick_3.csv
training complete
Data/Training\190722_tension_test_space_1_trial_2.csv
Da

Test model

In [43]:
start = time.time()
Expectation = []
Truth = []

with torch.no_grad():
    for path_idx in os.listdir(data_path_train):
        cur_path = os.path.join(data_path_train, path_idx)
        print(cur_path)
        input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
        # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
               
        inputs = torch.Tensor(input_file)
        inputs = inputs[:len(inputs)-len(inputs)%40]
        inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
        inputs = inputs.view([-1,40,9])
        
        test_loader = torch.utils.data.DataLoader(dataset=inputs,
                                       batch_size=batch_size, 
                                       shuffle=False)
        
        for inputs in test_loader:

            inputs, labels = torch.split(inputs, [8,1], 2)
            inputs = inputs.to(device)
            labels = labels[:, : ,:].view([-1,1])
            outputs = lstm(inputs)
            outputs = outputs[:,:,:].view([-1,1])
            
            labels_list = labels.tolist()
            outputs_list = outputs.tolist()
            
            zip_lists = zip(labels_list, outputs_list)
            for labels, outputs in zip_lists:
                Truth.append(labels)
                Expectation.append(outputs)

#             for i in range(inputs.size(1)):
#                 outputs = lstm(inputs[:,i,:].view([inputs.size(0),1,8]))
#                 total_outputs += outputs.cpu()           
            
        break

    print("This is Expectation size", len(Expectation))
    rmse = mean_squared_error(Expectation, Truth, squared = False)
    mae = mean_absolute_error(Expectation, Truth)
    
    print('Test Accuracy of the model RMSE: {}'.format(rmse))
    print('Test Accuracy of the model MAE: {}'.format(mae)) 
    print('time :', time.time() - start)
    


Data/Training\190722_tension_test_space_1_trial_2.csv
This is Expectation size 14040
Test Accuracy of the model RMSE: 1.930616034413703
Test Accuracy of the model MAE: 1.3391002100243623
time : 0.26728153228759766


Save model

In [44]:
torch.save(lstm.state_dict(), 'model.ckpt_Hochang_epochoutside')
print("save complete")

save complete


In [29]:
plt.plot(Expectation)
plt.plot(Truth)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.savefig('fig1.png', dpi=300)

In [ ]:
print(total_labels)